In [1]:
with open("input.txt") as f:
    instructions = [line.strip() for line in f.readlines()]

## part1

In [2]:
from collections import namedtuple

# InstructionRun stores results of running a set
# of an instructions. It holds the value of the
# accumulator, a list of booleans for whether or
# not a given instruction was executed (indexed
# by the instruction pointer), and whether or
# not the instructions terminated (this is False
# if it goes into an infinite loop).
InstructionRun = namedtuple(
    "InstructionRun",
    ["accumulator", "executed", "terminated"],
)

In [3]:
from typing import List

def run_instructions(
    instructions: List[str]
) -> InstructionRun:
    """Runs instructions and returns an InstructionRun
    object with the value of the accumulator, the
    instructions that were executed, and whether or not
    the instructions terminated.
    """
    executed = [False] * len(instructions)
    accumulator = 0
    inst_ptr = 0

    while inst_ptr < len(instructions) and not executed[inst_ptr]:
        instruction = instructions[inst_ptr]
        executed[inst_ptr] = True

        inst, count = instruction.split()
        count = int(count)

        if inst == "acc":
            accumulator += count
            inst_ptr += 1
        elif inst == "jmp":
            inst_ptr += count
        else:
            inst_ptr += 1

    return InstructionRun(
        accumulator, executed, inst_ptr == len(instructions)
    )


results = run_instructions(instructions)
results.accumulator

1749

## part2

We want to figure out which jmp/nop we can flip
to get the instructions to terminate. We check
all of the jmp/nop instructions that we executed
and try flipping them. If flipping them gets to
an instruction that we haven't already executed
then we consider it a possible solution.

We take all possible solutions and try re-running
the instructions to completion with the flipped
instruction and, if it terminates, then we've
found our solution.

In [4]:
possible = set()
for inst_ptr, instruction in enumerate(instructions):
    if not results.executed[inst_ptr]:
        continue

    inst, count = instruction.split()
    count = int(count)

    if inst == "jmp":
        if not results.executed[inst_ptr+1]:
            possible.add(inst_ptr)
    if inst == "nop":
        if not results.executed[inst_ptr+count]:
            possible.add(inst_ptr)

In [5]:
from copy import copy

def flip_instruction(instruction: str) -> str:
    """Flip jmp to nop or vice versa"""
    inst, count = instruction.split()
    flipped_inst = "jmp" if inst == "nop" else "nop"
    return f"{flipped_inst} {count}"

for inst_ptr in possible:
    inst = instructions[inst_ptr]

    instructions_copy = copy(instructions)
    instructions_copy[inst_ptr] = flip_instruction(inst)

    results = run_instructions(instructions_copy)
    if results.terminated:
        print(results.accumulator)
        break

515
